# Importing Libraries

In [15]:
import cv2
import numpy as np
import ftfy
import pytesseract
from PIL import Image
import regex
from fuzzywuzzy import fuzz

# OCR

In [16]:
def OCR(imgpath):
    img = cv2.imread(imgpath)
    img = cv2.resize(img, None, fx=2, fy=2,interpolation=cv2.INTER_CUBIC)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    pytesseract.pytesseract.tesseract_cmd = 'C:/Program Files/Tesseract-OCR/tesseract.exe'
    text = pytesseract.image_to_string(img, lang = 'eng+hin')
    text = ftfy.fix_text(text)
    text = ftfy.fix_encoding(text)
    return text

# Fuzzy substring match

In [17]:
def fuzzMatch( major, minor, errs = 4, threshold = 65 ):
    errs_ = 0
    major = major.lower()
    minor = minor.lower()
    s = regex.search(f"({minor}){{e<={errs_}}}", major)
    while s is None and errs_ <= errs:
        errs_ += 1
        s = regex.search(f"({minor}){{e<={errs_}}}", major)
    if(fuzz.token_set_ratio(minor,s)>threshold):
        return 1
    else:
        return 0

# Defining identification words

In [18]:
idKeyWords = {"PAN":["income","permanent","account"],"DL":["driving","drive","dl no","transport"], "Passport": ["republic","passport"],"Aadhaar":["aadhaar"]}

# Classification

In [19]:
def classify(text):
    for key in idKeyWords:
        for word in idKeyWords[key]:
            if(fuzzMatch(text,word)):
                return key
    return "other"    
    

In [20]:
def segregate(filepaths):
    classifiedDict = {"PAN":[],"DL":[],"Passport":[],"Aadhaar":[],"other":[]}
    for path in filepaths:
        ocrTxt = OCR(path)
        doctype = classify(ocrTxt)
        if(doctype=="other"):
            res = ocrTxt.split()
            for i in range(len(res) - 1):
                check = 1
                for j in range(3):
                    check = check and len(res[i+j]) == 4 and res[i+j].isdigit()
                check = check and ("male" in res or "MALE" in res)
                if(check):
                    doctype = "Aadhaar"
        classifiedDict[doctype].append(path)
    return classifiedDict

In [21]:
filepaths = ["aadhar.png","aadharlk.png","aadharlkb.png","image.jpg"]

In [22]:
classifieddict = segregate(filepaths)

In [23]:
classifieddict

{'PAN': ['image.jpg'],
 'DL': [],
 'Passport': [],
 'Aadhaar': ['aadhar.png', 'aadharlk.png', 'aadharlkb.png'],
 'other': []}

In [24]:
txt = OCR("aadhar.png")

In [25]:
fuzzMatch(txt,"aadhaar")

1

In [26]:
classify(txt)

'Aadhaar'

In [27]:
txt

'be\nGovemment of ind\n\nब आधार पहचान का प्रमाण है, नागरिकता का नहीं।\n\n# आधार विशिष्ट और सुरक्षित है।\n\n1 सुरक्षित FR कोइ/ऑफ़लाइन एक्सएमएल्/ऑनलाइन प्रमाणीकरण\nउपयोग करके पहचान सत्यापित करें।\n\nGovernment of India\n\n1 ह आधार के सभी रुप जैसे आधार पत्र, पीवीसी कार्ड, ई-आधार और एम-\n\' आधार समान रूप से मान्य हैं। १२ अंकों की आधार संख्या के स्थान पर\n1 arr (वर्चुअल) आधार पहचान (VID) का भी उपयोग किया जा\n\' सकता है।\n\n\' © to साल मैं कम से कम एक बार आधार अपडेट जरुर करें।\n\n\' Booey आपको विभिन्\u200dन सरकारी और गैर-सरकारी योजनाओं /सेवाओं\n\nभारतीय विशिष्ट पहचान प्राधिकरण\nUnique Identification Authority of India\nनामांकन क्रम/ Enrolment No.: 0000/00735/85861\n\nto we का लाभ उठाने मैं मदद करता है।\n\nने ह आधार में अपना मोबाइल नंबर और ई-मेल आईडी अपडेट रखें।\n\nLt ae laure © आधार सेवाओं का ATH उठाने के लिए स्मार्टफोन पर mAadhaar ऐप\nडाउनलोड करें।\n\nHOUSE NO. -26\n\nGANGA VIHAR & आधार/बायोमेट्रिक्स को लॉक/अनलॉक करने की विशेषता का उपयोग\n\nNEAR KAROLI MATA MANDIR सुरक्षा सुनिश्चित करने के 

In [28]:
for key in idKeyWords:
        for word in idKeyWords[key]:
            print(word + "\n")

income

permanent

account

driving

drive

dl no

transport

republic

passport

aadhaar

